<a href="https://colab.research.google.com/github/mtsizh/bottleneck-distance-for-sigma8/blob/main/wasserstein_distance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title #Download datasets
#@markdown Run this block of code to download our dataset to Google Colab.
#@markdown If successful this code yields 
#@markdown * folder `alldata2` containing all data for further scripts
#@markdown * file `alldata_large.zip` -- the same data zipped; not needed for scripts, but you may want to download it to your local machine

from IPython.display import clear_output

files = ['alldata.zip', 'alldata.z01', 'alldata.z02', 'alldata.z03']
print('deleting old files if exist')
!rm -rf alldata2
!rm -f alldata_large.zip
for f in files:
  !rm -f {f}
for i,f in enumerate(files):
  print('downloading part ', i, ' of ', len(files))
  #result = !curl -s -S https://raw.githubusercontent.com/mtsizh/bottleneck-distance-for-sigma8/main/{f} > /dev/null && echo "TRUE"
  result = !wget -q https://raw.githubusercontent.com/mtsizh/bottleneck-distance-for-sigma8/main/{f} && echo "1" || echo "0"
  if result == ['1']:
    print('OK')
  else:
    raise Exception("ERROR WHILE DOWNLOADING DATA")
clear_output()
print("DOWNLOAD SUCCESSFUL")
print("PARTS TO ZIP")
result = !zip -qq -F alldata.zip --out alldata_large.zip 2>/dev/null && echo "1" || echo "0"
if result == ['1']:
  print('OK')
else:
  raise Exception("ERROR WHILE CREATING LARGE ZIP")
print("UNZIPPING")
result = !unzip -o -qq alldata_large.zip 2>/dev/null && echo "1" || echo "0"
if result == ['1']:
  print('OK')
else:
  raise Exception("ERROR WHILE UNZIPPING")
print('cleaning')
for f in files:
  !rm -f {f}
clear_output()
print("ALL DATA DOWNLOADED AND UNPACKED")

ALL DATA DOWNLOADED AND UNPACKED


install gudhi library with the following code

In [2]:
!pip install gudhi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.6/31.6 MB 32.8 MB/s eta 0:00:00


In [18]:
#@markdown Folder with data (should be ok by default)
root_folder = "./alldata2" #@param {type: "string"}
#@markdown Choose filtering in % (0 to 100)
remove_masses_lower_than = 0  #@param {type: "slider", min: 0, max: 100}
remove_masses_higher_than = 100  #@param {type: "slider", min: 0, max: 100}
bootstrap_percent = 100  #@param {type: "slider", min: 0, max: 100}

if remove_masses_higher_than <= remove_masses_lower_than:
  rise("wrong filtering criteria")

import os
from glob import glob
import re
import numpy as np
import pandas as pd
from functools import lru_cache
from ipywidgets import IntProgress
from IPython import display


def read_all_data(filename, bootstrap_percent=100, mass_filter=[0, 100]):
  df = pd.read_csv(filename, sep=r'\t', header=None, engine ='python')
  df.drop(df.columns.difference([0,1,2,3]), axis=1, inplace=True)
  df.rename(columns={0:'x', 1:'y', 2:'z', 3:'m'}, inplace=True)
  #x = df.apply(lambda s: pd.to_numeric(s, errors='coerce').notnull().all())
  min_m = df['m'].quantile(mass_filter[0]/100)
  max_m = df['m'].quantile(mass_filter[1]/100)
  df = df[df['m'].between(min_m, max_m)]
  df.drop(['m'], axis=1, inplace=True)
  #print(df['m'].quantile(mass_filter[0]/100))
  return df.to_numpy()

@lru_cache(None)
def get_simplex_tree(point_set):
  alpha_complex = gudhi.AlphaComplex(points=point_set)
  simplex_tree = alpha_complex.create_simplex_tree(default_filtration_value=False)
  simplex_tree.compute_persistence()
  return simplex_tree

def get_persistence_intervals(point_set, dim):
  persistence_intervals = simplex_tree.persistence_intervals_in_dimension(dim)
  return persistence_intervals

dat_files = [y for x in os.walk(root_folder) for y in glob(os.path.join(x[0], '*.dat'))]
reg_expr = '.*\/([0-9]+)groups_([0-9]+)_new.dat'
parsed = [{'filename': f, 
           'sigma': int(re.match(reg_expr, f).group(1)), 
           'red_shift': int(re.match(reg_expr, f).group(2))
           } for f in dat_files]
all_data = pd.DataFrame(columns=["sigma8*10", "red_shift", "dimension", "born", "persists"])

bar = IntProgress(min=0, max=len(files))
bar.value = 0
display.display(bar)

for f in parsed:
  print(f)
  data = read_all_data(f['filename'])
  #for dim in [0, 1, 2]:
  #  I = get_persistence_intervals(data, dim)
  #  df = {'sigma8*10': f['param'], 
  #        'red_shift': f['red_shift'], 
  #        'dimension': dim,
  #        'born': I[:,0].tolist(),
  #        'persists': (I[:,1] - I[:,0]).tolist()}
  #  all_data = all_data.append(df, ignore_index = True)
  bar.value += 1 

#data = all_data.groupby(['sigma8*10', 'red_shift', 'dimension']).agg(lambda x: sum(list(x), [])).reset_index()
#data.to_json('diffs.json')
print("complete!")

IntProgress(value=0, max=4)

{'filename': './alldata2/z01/var6/05groups_20_new.dat', 'sigma': 5, 'red_shift': 20}
             x         y         z             m
0     0.340817  0.339700  0.332249  3.174038e+12
1     0.271638  0.289840  0.209213  2.946672e+12
2     0.315419  0.127297  0.956878  1.526360e+12
3     0.392967  0.476963  0.205095  1.222431e+12
4     0.967784  0.838600  0.533131  1.066466e+12
...        ...       ...       ...           ...
6393  0.041625  0.812007  0.751034  9.591905e+08
6394  0.037720  0.460073  0.605869  9.591905e+08
6395  0.028333  0.783743  0.565585  9.591905e+08
6396  0.021834  0.943464  0.370889  9.591905e+08
6397  0.021316  0.742524  0.483540  9.591905e+08

[6398 rows x 4 columns]
{'filename': './alldata2/z01/var6/10groups_20_new.dat', 'sigma': 10, 'red_shift': 20}
             x         y         z             m
0     0.292484  0.306584  0.220834  6.246728e+12
1     0.312709  0.316194  0.284927  6.171259e+12
2     0.987141  0.831879  0.547732  5.913410e+12
3     0.511883  0.51